In [2]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, SimpleRNN, LSTM

In [3]:
df_fraud=pd.read_csv('../Data/preprocessed.csv')
df_credit=pd.read_csv('../Data/credit_card_clean.csv')

In [4]:
df_credit.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
df_fraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,age,ip_address,class,hour_of_day,day_of_week,...,country_United States,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Viet Nam,country_Virgin Islands (U.S.),country_Yemen,country_Zambia,country_Zimbabwe
0,2,2015-01-11 03:47:13,2015-02-21 10:03:37,0.310345,FGBQNDNBETFJJ,0.120690,880217484,0,10,5,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,2015-06-02 16:40:57,2015-09-26 21:32:16,0.220690,MKFUIVOHLJBYN,0.344828,2785906106,0,21,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8,2015-05-28 07:53:06,2015-08-13 11:53:07,0.262069,SCQGQALXBUQZJ,0.120690,356056736,0,11,3,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12,2015-01-10 06:25:12,2015-03-04 20:56:37,0.179310,MSNWCFEHKTIOY,0.017241,2985180352,0,20,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16,2015-02-03 13:48:23,2015-03-12 12:46:23,0.000000,FROZWSSWOHZBE,0.241379,578312545,0,12,3,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
def preprocess_fraud_data(df):
    df['signup_time'] = pd.to_datetime(df['signup_time'])
    df['purchase_time'] = pd.to_datetime(df['purchase_time'])
    df = df.drop(['signup_time', 'purchase_time', 'user_id', 'device_id'], axis=1)
    return df

df_fraud = preprocess_fraud_data(df_fraud)

In [7]:
# Impute missing values
imputer = SimpleImputer(strategy='mean')
df_fraud = pd.DataFrame(imputer.fit_transform(df_fraud), columns=df_fraud.columns)
df_credit = pd.DataFrame(imputer.fit_transform(df_credit), columns=df_credit.columns)

In [8]:
# Ensure target variables are binary
df_fraud['class'] = df_fraud['class'].astype(int)
df_credit['Class'] = df_credit['Class'].astype(int)

In [9]:
# Separate features and target for Fraud_Data
X_fraud = df_fraud.drop('class', axis=1)
y_fraud = df_fraud['class']

# Separate features and target for creditcard data
X_credit = df_credit.drop('Class', axis=1)
y_credit = df_credit['Class']

# Train-test split for Fraud_Data
X_fraud_train, X_fraud_test, y_fraud_train, y_fraud_test = train_test_split(X_fraud, y_fraud, test_size=0.3, random_state=42)

# Train-test split for creditcard data
X_credit_train, X_credit_test, y_credit_train, y_credit_test = train_test_split(X_credit, y_credit, test_size=0.3, random_state=42)

In [10]:
# Model training and evaluation function
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"Performance of {model_name}:")
    print(classification_report(y_test, y_pred))

    # Log model and metrics with MLflow
    with mlflow.start_run():
        mlflow.sklearn.log_model(model, model_name)
        mlflow.log_param("model_type", model_name)
        mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
        mlflow.log_metric("precision", precision_score(y_test, y_pred))
        mlflow.log_metric("recall", recall_score(y_test, y_pred))
        mlflow.log_metric("f1_score", f1_score(y_test, y_pred))

In [11]:
# Logistic Regression
log_reg_fraud = LogisticRegression()
train_and_evaluate_model(log_reg_fraud, X_fraud_train, y_fraud_train, X_fraud_test, y_fraud_test, "Logistic Regression")

Performance of Logistic Regression:


c:\Users\Abdulaziz\Desktop\10 Academy\Adey Innovations\week8\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Abdulaziz\Desktop\10 Academy\Adey Innovations\week8\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Abdulaziz\Desktop\10 Academy\Adey Innovations\week8\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

              precision    recall  f1-score   support

           0       0.90      1.00      0.95     35044
           1       0.00      0.00      0.00      3700

    accuracy                           0.90     38744
   macro avg       0.45      0.50      0.47     38744
weighted avg       0.82      0.90      0.86     38744



2024/10/23 17:44:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
c:\Users\Abdulaziz\Desktop\10 Academy\Adey Innovations\week8\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
# Logistic Regression
log_reg_credit = LogisticRegression()
train_and_evaluate_model(log_reg_credit, X_credit_train, y_credit_train, X_credit_test, y_credit_test, "Logistic Regression")

c:\Users\Abdulaziz\Desktop\10 Academy\Adey Innovations\week8\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Performance of Logistic Regression:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     84984
           1       0.79      0.63      0.70       134

    accuracy                           1.00     85118
   macro avg       0.89      0.81      0.85     85118
weighted avg       1.00      1.00      1.00     85118



2024/10/23 17:45:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [13]:
# Decision Tree
dt_fraud = DecisionTreeClassifier()
train_and_evaluate_model(dt_fraud, X_fraud_train, y_fraud_train, X_fraud_test, y_fraud_test, "Decision Tree")

Performance of Decision Tree:
              precision    recall  f1-score   support

           0       0.95      0.93      0.94     35044
           1       0.48      0.57      0.52      3700

    accuracy                           0.90     38744
   macro avg       0.72      0.75      0.73     38744
weighted avg       0.91      0.90      0.90     38744



2024/10/23 17:45:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [14]:
# Decision Tree
dt_credit = DecisionTreeClassifier()
train_and_evaluate_model(dt_credit, X_credit_train, y_credit_train, X_credit_test, y_credit_test, "Decision Tree")

Performance of Decision Tree:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     84984
           1       0.70      0.75      0.72       134

    accuracy                           1.00     85118
   macro avg       0.85      0.87      0.86     85118
weighted avg       1.00      1.00      1.00     85118



2024/10/23 17:46:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [15]:
# Random Forest
rf_credit = RandomForestClassifier()
train_and_evaluate_model(rf_credit, X_credit_train, y_credit_train, X_credit_test, y_credit_test, "Random Forest")

Performance of Random Forest:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     84984
           1       0.95      0.74      0.83       134

    accuracy                           1.00     85118
   macro avg       0.98      0.87      0.92     85118
weighted avg       1.00      1.00      1.00     85118



2024/10/23 17:53:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [16]:
# Random Forest
rf_fraud = RandomForestClassifier()
train_and_evaluate_model(rf_fraud, X_fraud_train, y_fraud_train, X_fraud_test, y_fraud_test, "Random Forest")

Performance of Random Forest:
              precision    recall  f1-score   support

           0       0.95      1.00      0.98     35044
           1       0.99      0.54      0.69      3700

    accuracy                           0.95     38744
   macro avg       0.97      0.77      0.83     38744
weighted avg       0.96      0.95      0.95     38744



2024/10/23 17:55:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [17]:
# Gradient Boosting
gb_fraud = GradientBoostingClassifier()
train_and_evaluate_model(gb_fraud, X_fraud_train, y_fraud_train, X_fraud_test, y_fraud_test, "Gradient Boosting")

Performance of Gradient Boosting:
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     35044
           1       0.88      0.01      0.02      3700

    accuracy                           0.91     38744
   macro avg       0.89      0.50      0.49     38744
weighted avg       0.90      0.91      0.86     38744



2024/10/23 17:57:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [18]:
# Gradient Boosting
gb_credit = GradientBoostingClassifier()
train_and_evaluate_model(gb_credit,  X_credit_train, y_credit_train, X_credit_test, y_credit_test, "Gradient Boosting")

Performance of Gradient Boosting:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     84984
           1       0.76      0.49      0.59       134

    accuracy                           1.00     85118
   macro avg       0.88      0.74      0.80     85118
weighted avg       1.00      1.00      1.00     85118



2024/10/23 18:07:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [19]:
# Multi-Layer Perceptron (MLP)
mlp_credit = MLPClassifier()
train_and_evaluate_model(mlp_credit, X_credit_train, y_credit_train, X_credit_test, y_credit_test, "MLP")

Performance of MLP:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     84984
           1       0.37      0.71      0.48       134

    accuracy                           1.00     85118
   macro avg       0.68      0.85      0.74     85118
weighted avg       1.00      1.00      1.00     85118



2024/10/23 18:08:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [20]:
# Multi-Layer Perceptron (MLP)
mlp_fraud = MLPClassifier()
train_and_evaluate_model(mlp_fraud, X_fraud_train, y_fraud_train, X_fraud_test, y_fraud_test, "MLP")

Performance of MLP:
              precision    recall  f1-score   support

           0       0.90      1.00      0.95     35044
           1       0.00      0.00      0.00      3700

    accuracy                           0.90     38744
   macro avg       0.45      0.50      0.47     38744
weighted avg       0.82      0.90      0.86     38744



c:\Users\Abdulaziz\Desktop\10 Academy\Adey Innovations\week8\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Abdulaziz\Desktop\10 Academy\Adey Innovations\week8\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Abdulaziz\Desktop\10 Academy\Adey Innovations\week8\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

In [21]:
# Neural Network model architectures and training
def train_nn_model(model, X_train, y_train, X_test, y_test, model_name):
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

    # Log model and metrics with MLflow
    with mlflow.start_run():
        mlflow.keras.log_model(model, model_name)
        loss, accuracy = model.evaluate(X_test, y_test)
        mlflow.log_metric("accuracy", accuracy)

In [22]:
# Reshape data for CNN and RNN models
X_fraud_train_cnn_rnn = X_fraud_train.values.reshape((X_fraud_train.shape[0], X_fraud_train.shape[1], 1))
X_fraud_test_cnn_rnn = X_fraud_test.values.reshape((X_fraud_test.shape[0], X_fraud_test.shape[1], 1))
X_credit_train_cnn_rnn = X_credit_train.values.reshape((X_credit_train.shape[0], X_credit_train.shape[1], 1))
X_credit_test_cnn_rnn = X_credit_test.values.reshape((X_credit_test.shape[0], X_credit_test.shape[1], 1))

In [23]:
# CNN
cnn_model_fraud = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_fraud_train_cnn_rnn.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(1, activation='sigmoid')
])
train_nn_model(cnn_model_fraud, X_fraud_train_cnn_rnn, y_fraud_train, X_fraud_test_cnn_rnn, y_fraud_test, "CNN")

c:\Users\Abdulaziz\Desktop\10 Academy\Adey Innovations\week8\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 123s 38ms/step - accuracy: 0.8275 - loss: 1799898.6250 - val_accuracy: 0.9045 - val_loss: 13492.3994
Epoch 2/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 85s 30ms/step - accuracy: 0.8271 - loss: 83090.2656 - val_accuracy: 0.9045 - val_loss: 3820.8132
Epoch 3/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 92s 33ms/step - accuracy: 0.8374 - loss: 3000.2295 - val_accuracy: 0.9045 - val_loss: 0.3221
Epoch 4/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 89s 31ms/step - accuracy: 0.9057 - loss: 0.3231 - val_accuracy: 0.9045 - val_loss: 0.3151
Epoch 5/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 90s 32ms/step - accuracy: 0.9051 - loss: 0.3138 - val_accuracy: 0.9045 - val_loss: 0.3151
Epoch 6/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 94s 33ms/step - accuracy: 0.9050 - loss: 0.3139 - val_accuracy: 0.9045 - val_loss: 0.3151
Epoch 7/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 96s 34ms/step - accuracy: 0.9032 - loss: 0.3180 - val_accuracy: 0.9045 - val_loss: 0.3152
Epoch 8/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 98s 35ms/step

2024/10/23 18:27:04 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/23 18:28:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


1211/1211 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.9063 - loss: 0.3111


In [24]:
# CNN Model
cnn_model_credit = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_credit_train_cnn_rnn.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Train and evaluate the CNN model
train_nn_model(cnn_model_credit, X_credit_train_cnn_rnn, y_credit_train, X_credit_test_cnn_rnn, y_credit_test, "CNN")

Epoch 1/10
6207/6207 ━━━━━━━━━━━━━━━━━━━━ 47s 7ms/step - accuracy: 0.9943 - loss: 6.8250 - val_accuracy: 0.9991 - val_loss: 0.1440
Epoch 2/10
6207/6207 ━━━━━━━━━━━━━━━━━━━━ 47s 8ms/step - accuracy: 0.9984 - loss: 0.3182 - val_accuracy: 0.9990 - val_loss: 0.0318
Epoch 3/10
6207/6207 ━━━━━━━━━━━━━━━━━━━━ 48s 8ms/step - accuracy: 0.9990 - loss: 0.0247 - val_accuracy: 0.9994 - val_loss: 0.0186
Epoch 4/10
6207/6207 ━━━━━━━━━━━━━━━━━━━━ 48s 8ms/step - accuracy: 0.9991 - loss: 0.0232 - val_accuracy: 0.9990 - val_loss: 0.0058
Epoch 5/10
6207/6207 ━━━━━━━━━━━━━━━━━━━━ 49s 8ms/step - accuracy: 0.9992 - loss: 0.0096 - val_accuracy: 0.9993 - val_loss: 0.0077
Epoch 6/10
6207/6207 ━━━━━━━━━━━━━━━━━━━━ 48s 8ms/step - accuracy: 0.9991 - loss: 0.0101 - val_accuracy: 0.9994 - val_loss: 0.0045
Epoch 7/10
6207/6207 ━━━━━━━━━━━━━━━━━━━━ 52s 8ms/step - accuracy: 0.9991 - loss: 0.0061 - val_accuracy: 0.9993 - val_loss: 0.0194
Epoch 8/10
6207/6207 ━━━━━━━━━━━━━━━━━━━━ 50s 8ms/step - accuracy: 0.9991 - loss: 0

2024/10/23 18:37:14 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/23 18:37:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


2660/2660 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.9994 - loss: 0.0030


In [25]:
# RNN
rnn_model_fraud = Sequential([
    SimpleRNN(100, activation='relu', input_shape=(X_fraud_train_cnn_rnn.shape[1], 1)),
    Dense(1, activation='sigmoid')
])
train_nn_model(rnn_model_fraud, X_fraud_train_cnn_rnn, y_fraud_train, X_fraud_test_cnn_rnn, y_fraud_test, "RNN")

c:\Users\Abdulaziz\Desktop\10 Academy\Adey Innovations\week8\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 211s 73ms/step - accuracy: 0.9040 - loss: 0.3584 - val_accuracy: 0.9045 - val_loss: 0.3151
Epoch 2/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 205s 73ms/step - accuracy: 0.9052 - loss: 0.3147 - val_accuracy: 0.9045 - val_loss: 0.3157
Epoch 3/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 205s 73ms/step - accuracy: 0.9059 - loss: 0.3128 - val_accuracy: 0.9045 - val_loss: 0.3167
Epoch 4/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 203s 72ms/step - accuracy: 0.9051 - loss: 0.3148 - val_accuracy: 0.9045 - val_loss: 0.3157
Epoch 5/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 204s 72ms/step - accuracy: 0.9041 - loss: 0.3169 - val_accuracy: 0.9045 - val_loss: 0.3151
Epoch 6/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 251s 68ms/step - accuracy: 0.9046 - loss: 0.3155 - val_accuracy: 0.9045 - val_loss: 0.3151
Epoch 7/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 202s 72ms/step - accuracy: 0.9045 - loss: 0.3158 - val_accuracy: 0.9045 - val_loss: 0.3151
Epoch 8/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 204s 72ms/step - accuracy: 

2024/10/23 19:12:54 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/23 19:14:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


1211/1211 ━━━━━━━━━━━━━━━━━━━━ 27s 22ms/step - accuracy: 0.9063 - loss: 0.3118


In [26]:
# RNN Model
rnn_model_credit = Sequential([
    SimpleRNN(100, activation='relu', input_shape=(X_fraud_train_cnn_rnn.shape[1], 1)),
    Dense(1, activation='sigmoid')
])

# Train and evaluate the RNN model using reshaped fraud data
train_nn_model(rnn_model_credit, X_fraud_train_cnn_rnn, y_fraud_train, X_fraud_test_cnn_rnn, y_fraud_test, "RNN")

Epoch 1/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 210s 73ms/step - accuracy: 0.9014 - loss: 0.3624 - val_accuracy: 0.9045 - val_loss: 0.3160
Epoch 2/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 204s 72ms/step - accuracy: 0.9046 - loss: 0.3156 - val_accuracy: 0.9045 - val_loss: 0.3151
Epoch 3/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 204s 72ms/step - accuracy: 0.9047 - loss: 0.3154 - val_accuracy: 0.9045 - val_loss: 0.3153
Epoch 4/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 211s 75ms/step - accuracy: 0.9064 - loss: 0.3120 - val_accuracy: 0.9045 - val_loss: 0.3156
Epoch 5/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 203s 72ms/step - accuracy: 0.9043 - loss: 0.3166 - val_accuracy: 0.9045 - val_loss: 0.3151
Epoch 6/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 203s 72ms/step - accuracy: 0.9042 - loss: 0.3164 - val_accuracy: 0.9045 - val_loss: 0.3153
Epoch 7/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 200s 71ms/step - accuracy: 0.9052 - loss: 0.3142 - val_accuracy: 0.9045 - val_loss: 0.3151
Epoch 8/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 198s 70ms/step - accuracy: 

2024/10/23 19:48:27 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/23 19:49:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


1211/1211 ━━━━━━━━━━━━━━━━━━━━ 27s 22ms/step - accuracy: 0.9063 - loss: 0.3110


In [27]:
# LSTM
lstm_model_credit = Sequential([
    LSTM(100, activation='relu', input_shape=(X_credit_train_cnn_rnn.shape[1], 1)),
    Dense(1, activation='sigmoid')
])
train_nn_model(lstm_model_credit, X_credit_train_cnn_rnn, y_credit_train, X_credit_test_cnn_rnn, y_credit_test, "LSTM")

Epoch 1/10
6207/6207 ━━━━━━━━━━━━━━━━━━━━ 197s 31ms/step - accuracy: 0.9572 - loss: 47.2489 - val_accuracy: 0.9984 - val_loss: 0.1125
Epoch 2/10
6207/6207 ━━━━━━━━━━━━━━━━━━━━ 189s 30ms/step - accuracy: 0.9954 - loss: 37.9891 - val_accuracy: 0.9984 - val_loss: 155.1720
Epoch 3/10
6207/6207 ━━━━━━━━━━━━━━━━━━━━ 193s 31ms/step - accuracy: 0.9963 - loss: 36.2267 - val_accuracy: 0.9984 - val_loss: 2.0894
Epoch 4/10
6207/6207 ━━━━━━━━━━━━━━━━━━━━ 192s 31ms/step - accuracy: 0.9959 - loss: 8.7120 - val_accuracy: 0.9984 - val_loss: 11.3192
Epoch 5/10
6207/6207 ━━━━━━━━━━━━━━━━━━━━ 201s 32ms/step - accuracy: 0.9963 - loss: 33.7974 - val_accuracy: 0.9984 - val_loss: 83.7473
Epoch 6/10
6207/6207 ━━━━━━━━━━━━━━━━━━━━ 192s 31ms/step - accuracy: 0.9937 - loss: 139.8244 - val_accuracy: 0.9983 - val_loss: 14.9109
Epoch 7/10
6207/6207 ━━━━━━━━━━━━━━━━━━━━ 196s 32ms/step - accuracy: 0.9951 - loss: 37.5117 - val_accuracy: 0.9984 - val_loss: 40.1457
Epoch 8/10
6207/6207 ━━━━━━━━━━━━━━━━━━━━ 192s 31ms/step

2024/10/23 20:22:22 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/23 20:23:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


2660/2660 ━━━━━━━━━━━━━━━━━━━━ 28s 10ms/step - accuracy: 0.9984 - loss: 68.9116


In [28]:
# LSTM Model
lstm_model_fraud = Sequential([
    LSTM(100, activation='relu', input_shape=(X_fraud_train_cnn_rnn.shape[1], 1)),
    Dense(1, activation='sigmoid')
])

# Train and evaluate the LSTM model using reshaped fraud data
train_nn_model(lstm_model_fraud, X_fraud_train_cnn_rnn, y_fraud_train, X_fraud_test_cnn_rnn, y_fraud_test, "LSTM")

Epoch 1/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 571s 200ms/step - accuracy: 0.6992 - loss: 344711397376.0000 - val_accuracy: 0.0955 - val_loss: 178050.0156
Epoch 2/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 615s 198ms/step - accuracy: 0.8234 - loss: 111136.1016 - val_accuracy: 0.9045 - val_loss: 238644.4688
Epoch 3/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 558s 197ms/step - accuracy: 0.8149 - loss: 140065264.0000 - val_accuracy: 0.9045 - val_loss: 17603.2930
Epoch 4/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 551s 195ms/step - accuracy: 0.8192 - loss: 4013503.5000 - val_accuracy: 0.9045 - val_loss: 270101.3125
Epoch 5/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 537s 190ms/step - accuracy: 0.8280 - loss: 133066.8594 - val_accuracy: 0.0955 - val_loss: 625788.7500
Epoch 6/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 537s 190ms/step - accuracy: 0.8155 - loss: 31996768.0000 - val_accuracy: 0.9045 - val_loss: 34856852.0000
Epoch 7/10
2826/2826 ━━━━━━━━━━━━━━━━━━━━ 534s 189ms/step - accuracy: 0.8284 - loss: 9366419.0000 - val_accuracy: 0.9045 - 

2024/10/23 21:56:05 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/10/23 21:57:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


1211/1211 ━━━━━━━━━━━━━━━━━━━━ 77s 63ms/step - accuracy: 0.9063 - loss: 9114.0830


In [30]:
run_id = mlflow.active_run().info.run_id
print(f"Run ID: {run_id}")

Run ID: None


In [34]:
# saved best performing model as fraud_model.pkl
import pickle

def save_model(model, filename):
  pickle.dump(model, open(filename, 'wb'))

In [35]:
save_model(rf_fraud, "../Models/fraud_model.pkl")

In [36]:
save_model(rf_credit, '../Models/credit_model.pkl')